In [1]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools
import cudf
print('We will use RAPIDS version',cudf.__version__)
cudf.set_option("default_integer_bitwidth", 32)
cudf.set_option("default_float_bitwidth", 32)


from coVisitation import CV_B2B, CV_toClicks, CV_carts_orders, ClicktoClick, ClicktoCart, ClicktoOrder

We will use RAPIDS version 22.10.01+2.gca9a422da9


In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'
! mkdir {outputPath}

mkdir: cannot create directory ‘../output/newSplited/’: File exists


# Calculate covisitaion matrix

In [3]:
SET = 2
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [4]:
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(type_labels).astype('int8')
    return df

In [6]:
data_cache = {}
if SET == 1:
    files = glob.glob('../input/split_2-1_pqt/*_parquets/*')
    for f in files: 
        data_cache[f] = read_file_to_cache(f)
elif SET == 2:
    files = glob.glob('../input/splited/parquets/*_parquets/*')
    for f in files: 
        data_cache[f] = pd.read_parquet(f)
elif SET == 3:
    files = glob.glob('../input/parquets/*_parquets/*')
    for f in files: 
        data_cache[f] = read_file_to_cache(f)

saveFolder = outputPath + f'coVisit_set{SET}_top20_20_20/'
try: 
    os.makedirs(saveFolder) 
except OSError as error: 
    print(error)

[Errno 17] File exists: '../output/newSplited/coVisit_set2_top20_20_20/'


In [11]:
CovisitMat = [ClicktoOrder, ClicktoCart, ClicktoClick, CV_toClicks, CV_carts_orders, CV_B2B]
topN_covisit = {ClicktoOrder: 20, ClicktoCart: 20, ClicktoClick: 20, CV_toClicks: 20, CV_carts_orders: 20, CV_B2B: 20}

In [ ]:
for covisitMat in CovisitMat:
    covisiter = covisitMat(files, data_cache, saveFolder)
    covisiter.processDisks(topN_covisit[covisitMat])